In [1]:
import pandas as pd
import os
import psycopg2
import boto3

### Upload to S3

In [2]:
bucket = os.environ["AWS_S3_SCRATCH_SPACE"]
iamrole = os.environ["STS_ROLE_ARN"]

def upload_s3(file,bucket):
    client = boto3.client('s3',\
                          aws_access_key_id = os.environ["AWS_ACCESS_KEY_ID"],\
                          aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"]
                         )
    return client.upload_file(file,bucket,file)

In [3]:
files = ['campaign.csv','reward_campaign.csv','reward_transaction.csv']

for i in range(3):
    upload_s3(files[i],bucket)
    print(f'''{files[i]} has been successfully uploaded to S3''')

campaign.csv has been successfully uploaded to S3
reward_campaign.csv has been successfully uploaded to S3
reward_transaction.csv has been successfully uploaded to S3


### Load to Redshift

In [4]:
def s3_to_redshift(tbl,file):
    conn = psycopg2.connect(
        host=os.environ["AWS_REDSHIFT_HOST"],
        user=os.environ["AWS_REDSHIFT_USER"],
        port=os.environ["AWS_REDSHIFT_PORT"],
        password=os.environ["AWS_REDSHIFT_PASSWORD"],
        dbname=os.environ["DATABASE"]
        )
    cur = conn.cursor()
    cur.execute(f'''truncate table {tbl};''')
    print(f"{tbl} Successfully Truncated")
    cur.execute(f'''copy {tbl} from 's3://{bucket}/{file}' iam_role '{iamrole}' 
                ignoreheader 1
                delimiter ','
                timeformat 'auto'
                dateformat 'auto';''')
    conn.commit()
    conn.close()
    print(f"{tbl} Successfully Loaded")

In [7]:
for i in range(3):
    table = files[i][:-4]
    file = files[i]
    s3_to_redshift(table,file)

campaign Successfully Truncated
campaign Successfully Loaded
reward_campaign Successfully Truncated
reward_campaign Successfully Loaded
reward_transaction Successfully Truncated
reward_transaction Successfully Loaded
